# Objetivo

Criar um modelo de machine learning para fazer a previsão de inadimplencia de clientes

In [ ]:
# Manipulação de dados
import pandas as pd
import numpy as np
import redshift_connector

# Converter dados para date
from datetime import datetime

# Modelos de Machine Learning
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn import svm
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB

from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.metrics import classification_report, log_loss



from sklearn.preprocessing import MinMaxScaler
from sklearn import preprocessing
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
# Vizualização de dados
import matplotlib.pyplot as plt
import seaborn as sns

# Balanceamento das classes
from imblearn.over_sampling import SMOTE
#from imblearn.under_sampling import NearMiss
#import imblearn

In [ ]:
# Parando as mensagens de warnings
import sys
import warnings
if not sys.warnoptions:
    warnings.simplefilter("ignore")

# Sumario

1 - Carregando o Dataset
2 - SQL utilizada para retirar os dados do BD IBM
3 - Convertendo os tipos dos dados
4 - Análise exploratória
5 - Correlação dos Dados
6 - Transformações nos dados e balanceamento de classes
7 - Testando vários modelos de classificação
8 - Procedimentos para entregar novos dados


## 1 - Carregando o Dataset

In [ ]:
# Carregando o arquivo CSV dos dados que foram exportados do redshift

dataset = pd.read_csv("d_clients_mv4.csv")
dataset_BTYD = pd.read_csv("BTYD_probs.csv")

In [ ]:
# Passando o arquivo CSV para Dataframe 

df = pd.DataFrame(dataset)

In [ ]:
df.shape

In [ ]:
df['risco_credito'].value_counts()

In [ ]:
df['churn_class'].value_counts()

In [ ]:
# Verificando os tipos dos dados

df.info()

In [ ]:
df.shape

## 3 - Convertendo os tipos dos dados

In [ ]:
# Tipo object -  Variáveis convertidas para objects
cols_objects = ['cd_cliente','ds_tp_pessoa','risco_credito'
               ,'churn_class']
df[cols_objects] = df[cols_objects].astype(object)

# Tipo Inteiro -  Convertendo as variáveis do tipo inteiro
cols_ints = ['pedidos','recencia','churn_class_number','soma_qt_cancelada','soma_qt_pendente']
df[cols_ints] = df[cols_ints].astype(int)

# Tipo float -  Convertendo as variáveis do tipo float
cols_float = ['saldo_credev','saldo_adiantamento','score_empresas','negativacoes','soma_vlr_cancelado','soma_vlr_pendente']
df[cols_float] = df[cols_float].astype(float)

# Tipo Date -  Convertendo variáveis datetime - não vamos usar dados temporais neste modelo
df['ano_mes_dt_cad_rep'] = df['ano_mes_dt_cad_rep'] = pd.to_datetime(df['ano_mes_dt_cad_rep'], format='%Y-%m')

In [ ]:
# Criando uma lista das variáveis numéricas para posterior análise

cols_numerics = ['pedidos','recencia','churn_class_number','soma_qt_cancelada','soma_qt_pendente',
                'saldo_credev','saldo_adiantamento','score_empresas','negativacoes','soma_vlr_cancelado','soma_vlr_pendente']

In [ ]:
# Verificando novamente os tipos dos dados
# Agora todos as colunas estão com os tipos de dados corretos

df.dtypes

In [ ]:
# Categorização dummy
dummies = ['risco_credito', 'volume_compra', 'ds_tp_pessoa']


num_df = pd.get_dummies(df[dummies])
num_df

In [ ]:
df = pd.concat([df,num_df],axis=1)
df

## 4 - Análise exploratória

In [ ]:
# Verificando a proporção de classes churn
# Vai ser necessário equilivrar as classes para treinar o modelo
# A variável está codificada 0 para ativo recorrente 1 para em churn

df['churn_class'].value_counts()

In [ ]:
# Verificando o formato do dataset

df.shape

In [ ]:
df

## 5 - Correlação dos Dados

Analisamos a correlação entre as variáveis numéricas e entre as variáveis numéricas e a variável alvo (o que estamos querendo analisar).

In [ ]:
# Vai ser necessário filtrar as colunas de valor pois elas apresentam dados 
# colineares(colunas se tratando do mesmo tipo de dado)

df.corr("spearman")

In [ ]:
df.info()

In [ ]:
df = pd.concat([df,dataset_BTYD],axis=1)
df

In [ ]:
# Analisando as correlações com a TX churn existem algumas variáveis com baixa correlação
# Vou remover essas variáveis para deixar o modelo mais generalista
# Vou aproveitar e remover as variáveis que não vão entrar no modelo
# Pois o modelo pode ficar enviezado por essas variáveis fazerem parte da regra de negócio que classifica o churn
vars_baixa_corr = ['cd_cliente','ds_tp_pessoa','volume_compra','risco_credito','churn_class','recencia','ano_mes_dt_cad_rep','saldo_credev'
                  ,'frequency','recency','T','monetary_value','pred_num_trans_30','pred_trans_avg_value','CLV']
df_2 = df.drop(columns=vars_baixa_corr)

In [ ]:
# Verificando as colunas do novo dataset com as variáveis com uma correlação relativamente aceitavel
df_2 = df.dropna()
df_2

In [ ]:
# Passando a variável target para o tipo fator novamente

df_2['churn_class_number'] = df_2['churn_class_number'].astype(object)
df_2.info()

In [ ]:
# Verificando a distribuição das classes 0 e 1
df_2['churn_class_number'].value_counts()

## 6 - Transformações nos dados e balanceamento de classes

In [ ]:
# Normalização das variáveis numéricas, vou colocar elas todas na mesma escala
# Antes disso vou criar meu dataset de preditores e dataset de target   

preditores = ['saldo_adiantamento','score_empresas','negativacoes','pedidos','risco_credito_Risco Baixo',
             'risco_credito_Risco Considerável','risco_credito_Risco Iminente','risco_credito_Risco Muito Baixo',
             'risco_credito_Risco Médio','risco_credito_Risco Mínimo','risco_credito_Risco Relevante','volume_compra_ACIMA DE 90k',
             'volume_compra_ATÉ 10k','volume_compra_DE 10k A 30k','volume_compra_DE 30k A 60k'
              ,'ds_tp_pessoa_Fisica','ds_tp_pessoa_Juridica','soma_qt_cancelada','soma_qt_pendente','prob_alive']
target = ['churn_class_number']

X_ = df_2[preditores]
Y_ = df_2[target].astype(int)

In [ ]:
# Balanceamento de classes com SMOTE
smt = SMOTE()

X, Y = smt.fit_resample(X_, Y_)
Y = Y.astype(int)
# Nova distribuição das classes balanceadas
Y.value_counts()

In [ ]:
# Normalização dos dados

#min_max_scaler = preprocessing.MinMaxScaler()
#X = min_max_scaler.fit_transform(X)
#X_ = min_max_scaler.fit_transform(X_)

In [ ]:
# Agora vou fazer a divisão dos dados de treino e teste com proporção de 30% para teste
# Divisão em dados de treino e de teste
# Não irei balancear as classes para o modelo RF pois apresenta um desempenho mt bom sem o balanceamento

x_train, x_valid, y_train, y_valid = train_test_split(X, Y, test_size=0.3)
x_train_, x_valid_, y_train_, y_valid_ = train_test_split(X_, Y_, test_size=0.3)

In [ ]:
# Imprimindo o número de exemplos (observações) em cada dataset
print("Exemplos de Treino: {}".format(len(x_train)))
print("Exemplos de Validação: {}".format(len(y_valid)))
print("Exemplos de Teste: {}".format(len(y_train)))

## 7 - Testando vários modelos de classificação

In [ ]:
# Treinando vários modelos para teste

# https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html
# modelLR = LogisticRegression(penalty='none')
# modelLR.fit(x_train, y_train)

# https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html
modelRF = RandomForestClassifier(max_depth=8, max_features=10, min_samples_leaf=4,
                       min_samples_split=10)
modelRF.fit(x_train_, y_train_)

# https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.GradientBoostingClassifier.html
modelGB = GradientBoostingClassifier(loss='exponential', n_estimators=150)
modelGB.fit(x_train, y_train)

# https://scikit-learn.org/stable/modules/svm.html
# modelSVM = svm.SVC()
# modelSVM.fit(x_train, y_train)

# https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html
# modelKN = KNeighborsClassifier(n_neighbors=3)
# modelKN.fit(x_train, y_train)   
    
# https://scikit-learn.org/stable/modules/naive_bayes.html
# modelNB = GaussianNB()
# modelNB = modelNB.fit(x_train, y_train)

In [ ]:
# Os modelos RF e GB demonstraram melhor desemprenho dos demais, 
# por isso irei aplicar o RSearch no GB e RF para otimizar seus parametros
# https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.RandomizedSearchCV.html

# RF
# Definição dos parâmetros
#param_dist = {"max_depth": [1, 3, 7, 8, 12, None],
#              "max_features": [8, 9, 10, 11, 16, 22],
#              "min_samples_split": [8, 10, 11, 14, 16, 19],
#              "min_samples_leaf": [1, 2, 3, 4, 5, 6, 7],
#              "bootstrap": [True, False]}

# Para o classificador criado com ExtraTrees, testamos diferentes combinações de parâmetros
# rsearch = RandomizedSearchCV(modelRF, param_distributions = param_dist, n_iter = 35, return_train_score = True)  

# Aplicando o resultado ao conjunto de dados de treino e obtendo o score
# rsearch.fit(x_train_, y_train_)

# Resultados 
# rsearch.cv_results_

# Imprimindo o melhor estimador
# modelRF = rsearch.best_estimator_
# print (modelRF)

# Aplicando o melhor estimador para realizar as previsões
# y_pred = modelRF.predict(x_valid_)

# Confusion Matrix
# confusionMatrix = confusion_matrix(y_valid_, y_pred)
# print(confusionMatrix)

# Acurácia
# accuracy = accuracy_score(y_valid_, y_pred)
# print(accuracy)

In [ ]:
# Os modelos RF e GB demonstraram melhor desemprenho dos demais, 
# por isso irei aplicar o RSearch no GB e RF para otimizar seus parametros
# https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.RandomizedSearchCV.html

# GB
# Definição dos parâmetros
# param_dist = {"loss": ['deviance', 'exponential'],
#               "learning_rate": [0.1, 0.03, 0.5, 0.7],
#               "n_estimators": [50, 100, 150, 200],
#               "subsample": [1.0,3.0],
#               "criterion": ['friedman_mse', 'mse']}

# Para o classificador criado com ExtraTrees, testamos diferentes combinações de parâmetros
# rsearch = RandomizedSearchCV(modelGB, param_distributions = param_dist, n_iter = 35, return_train_score = True)  

# Aplicando o resultado ao conjunto de dados de treino e obtendo o score
# rsearch.fit(x_train_, y_train_)

# Resultados 
# rsearch.cv_results_

# Imprimindo o melhor estimador
# modelGB = rsearch.best_estimator_
# print (modelGB)

# Aplicando o melhor estimador para realizar as previsões
# y_pred = modelGB.predict(x_valid_)

# Confusion Matrix
# confusionMatrix = confusion_matrix(y_valid_, y_pred)
# print(confusionMatrix)

# Acurácia
# accuracy = accuracy_score(y_valid_, y_pred)
# print(accuracy)

In [ ]:
# predictLR = modelLR.predict(x_valid)
# print('Logistic Regression: \n', classification_report(predictLR, y_valid))
# print('Logistic Regression Accuracy: ', accuracy_score(predictLR, y_valid))

predictRF = modelRF.predict(x_valid_)
print('Random Forest Classifier : \n', classification_report(predictRF, y_valid_))
print('Random Forest Classifier Accuracy: ', accuracy_score(predictRF, y_valid_))

predictGB = modelGB.predict(x_valid)
print('Gradient Boost Classifier : \n', classification_report(predictGB, y_valid))
print('Gradient Boost Classifier Accuracy: ', accuracy_score(predictGB, y_valid))

# predictSVM = modelSVM.predict(x_valid)
# print('SVM : \n', classification_report(predictSVM, y_valid))
# print('SVM Accuracy: ', accuracy_score(predictSVM, y_valid))

# predictKN = modelKN.predict(x_valid)
# print('KN : \n', classification_report(predictKN, y_valid))
# print('KN Accuracy: ', accuracy_score(predictKN, y_valid))

# predictNB = modelNB.predict(x_valid)
# print('NB : \n', classification_report(predictNB, y_valid))
# print('NB Accuracy: ', accuracy_score(predictNB, y_valid))

In [ ]:
# Confusion Matrix do Modelo Final
# print ("Confusion matrix")
# print('LR',confusion_matrix(predictLR, y_valid))
print('RF',confusion_matrix(predictRF, y_valid_))
print('GB',confusion_matrix(predictGB, y_valid))
# print('SVM',confusion_matrix(predictSVM, y_valid))
# print('KN',confusion_matrix(predictKN, y_valid))
# print('NB',confusion_matrix(predictNB, y_valid))

## 8 - Procedimentos para entregar novos dados

In [ ]:
# Carregando o arquivo CSV com novos dados para previsão

novos_dados = pd.read_csv("novosdados_mv4.csv")

In [ ]:
novos_dados.shape

In [ ]:
# Passando o arquivo CSV para Dataframe 
df_novos_dados = pd.DataFrame(novos_dados)

In [ ]:
# Tipo object -  Variáveis convertidas para objects
cols_objects = ['cd_cliente','ds_tp_pessoa','risco_credito'
               ,'churn_class']
df_novos_dados[cols_objects] = df_novos_dados[cols_objects].astype(object)

# Tipo Inteiro -  Convertendo as variáveis do tipo inteiro
cols_ints = ['pedidos','recencia','churn_class_number']
df_novos_dados[cols_ints] = df_novos_dados[cols_ints].astype(int)

# Tipo float -  Convertendo as variáveis do tipo float
cols_float = ['saldo_credev','saldo_adiantamento','score_empresas','negativacoes']
df_novos_dados[cols_float] = df_novos_dados[cols_float].astype(float)

# Tipo Date -  Convertendo variáveis datetime - não vamos usar dados temporais neste modelo
df_novos_dados['ano_mes_dt_cad_rep'] = df_novos_dados['ano_mes_dt_cad_rep'] = pd.to_datetime(df_novos_dados['ano_mes_dt_cad_rep'], format='%Y-%m')

In [ ]:
# Categorização dummy
dummies = ['risco_credito', 'volume_compra', 'ds_tp_pessoa']


num_df_new = pd.get_dummies(df_novos_dados[dummies])
num_df_new

In [ ]:
df_3 = pd.concat([df_novos_dados,num_df_new],axis=1)
df_3

In [ ]:
df_3 = pd.concat([df_3,dataset_BTYD],axis=1)
df

In [ ]:
# Dropando valores NA
df_3 = df.dropna()

In [ ]:
df_3['churn_class'].value_counts()

In [ ]:
df_3

In [ ]:
# Colocando os dados no mesmo padrão dos dados de treino


preditores_new = ['saldo_adiantamento','score_empresas','negativacoes','pedidos','risco_credito_Risco Baixo',
             'risco_credito_Risco Considerável','risco_credito_Risco Iminente','risco_credito_Risco Muito Baixo',
             'risco_credito_Risco Médio','risco_credito_Risco Mínimo','risco_credito_Risco Relevante','volume_compra_ACIMA DE 90k',
             'volume_compra_ATÉ 10k','volume_compra_DE 10k A 30k','volume_compra_DE 30k A 60k'
              ,'ds_tp_pessoa_Fisica','ds_tp_pessoa_Juridica','soma_qt_cancelada','soma_qt_pendente','prob_alive']
X_new_2 = df_3[preditores_new]

In [ ]:
# Normalização dos dados
#min_max_scaler = preprocessing.MinMaxScaler()
#X_new = min_max_scaler.fit_transform(X_new)

In [ ]:
# Fazendo previsão com novos dados
# predictLR_2 = modelLR.predict(X_new)
# predictLR_2_prob = modelLR.predict_proba(X_new)
# print('Logistic Regression: \n', classification_report(predictLR_2, df_2['churn_class_number']))
# print('Logistic Regression Accuracy: ', accuracy_score(predictLR_2, df_2['churn_class_number']))

predictRF_2 = modelRF.predict(X_new_2)
predictRF_2_prob = modelRF.predict_proba(X_new_2)
print('Random Forest Classifier : \n', classification_report(predictRF_2, df_3['churn_class_number']))
print('Random Forest Classifier Accuracy: ', accuracy_score(predictRF_2, df_3['churn_class_number']))

predictGB_2 = modelGB.predict(X_new_2)
predictGB_2_prob = modelGB.predict_proba(X_new_2)
print('Gradient Boost Classifier : \n', classification_report(predictGB_2, df_3['churn_class_number']))
print('Gradient Boost Classifier Accuracy: ', accuracy_score(predictGB_2, df_3['churn_class_number']))

# predictSVM_2 = modelSVM.predict(X_new)
# print('SVM Classifier : \n', classification_report(predictSVM_2, df_2['churn_class_number']))
# print('SVM Classifier Accuracy: ', accuracy_score(predictSVM_2, df_2['churn_class_number']))

# predictKN_2 = modelKN.predict(X_new)
# predictKN_2_prob = modelKN.predict_proba(X_new)
# print('KN Classifier : \n', classification_report(predictKN_2, df_2['churn_class_number']))
# print('KN Classifier Accuracy: ', accuracy_score(predictKN_2, df_2['churn_class_number']))

# predictNB_2 = modelNB.predict(X_new)
# predictNB_2_prob = modelNB.predict_proba(X_new)
# print('NB Classifier : \n', classification_report(predictNB_2, df_2['churn_class_number']))
# print('NB Classifier Accuracy: ', accuracy_score(predictNB_2, df_2['churn_class_number']))

In [ ]:
# Criando as colunas de previsão de cada modelo no dataset
# df_2['predictLR_2'] = list(predictLR_2)
df_3['predictRF_2'] = list(predictRF_2)
df_3['predictGB_2'] = list(predictGB_2)
# df_2['predictSVM_2'] = list(predictSVM_2)
# df_2['predictKN_2'] = list(predictKN_2)
# df_2['predictNB_2'] = list(predictNB_2)

In [ ]:
# Criando as colunas de previsão probabilistica de cada modelo no dataset
# df_2['predictLR_2_prob'] = list(predictLR_2_prob)
df_2['predictRF_2_prob'] = list(predictRF_2_prob)
df_2['predictGB_2_prob'] = list(predictGB_2_prob)
# df_2['predictKN_2_prob'] = list(predictKN_2_prob)
# df_2['predictNB_2_prob'] = list(predictNB_2_prob)

In [ ]:
df_3

In [ ]:
df_3.to_csv('predictions_new_mv4.csv', index=False, encoding='utf-8')

In [ ]:
df_3

In [ ]:
df_3['churn_class'].value_counts()

In [ ]:
# print(df_2['predictLR_2'].value_counts())
print(df_3['predictRF_2'].value_counts())
print(df_3['predictGB_2'].value_counts())
# print(df_2['predictSVM_2'].value_counts())
# print(df_2['predictKN_2'].value_counts())
# print(df_2['predictNB_2'].value_counts())

In [ ]:
# Confusion Matrix do Modelo Final
# print ("Confusion matrix LR")
# print(confusion_matrix(df_2['churn_class_number'], predictLR_2))
print ("Confusion matrix RF")
print(confusion_matrix(df_3['churn_class_number'], predictRF_2))
print ("Confusion matrix GB")
print(confusion_matrix(df_3['churn_class_number'], predictGB_2))
# print ("Confusion matrix SVM")
# print(confusion_matrix(df_2['churn_class_number'], predictSVM_2))
# print ("Confusion matrix KN")
# print(confusion_matrix(df_2['churn_class_number'], predictKN_2))
# print ("Confusion matrix NB")
# print(confusion_matrix(df_2['churn_class_number'], predictNB_2))

In [ ]:
# Plot da confusion matrix
# ax = plt.subplot()
# sns.heatmap(confusion_matrix(df_2['churn_class_number'], predictLR_2), annot=True, fmt='g', ax=ax);

# labels, title and ticks
# ax.set_xlabel('Predicted labels');ax.set_ylabel('True labels'); 
# ax.set_title('Confusion Matrix LR'); 
# ax.xaxis.set_ticklabels(['0', '1']); ax.yaxis.set_ticklabels(['0', '1']);

In [ ]:
# Plot da confusion matrix
ax = plt.subplot()
sns.heatmap(confusion_matrix(df_3['churn_class_number'], predictRF_2), annot=True, fmt='g', ax=ax);

# labels, title and ticks
ax.set_xlabel('Predicted labels');ax.set_ylabel('True labels'); 
ax.set_title('Confusion Matrix RF'); 
ax.xaxis.set_ticklabels(['0', '1']); ax.yaxis.set_ticklabels(['0', '1']);

In [ ]:
# Plot da confusion matrix
ax = plt.subplot()
sns.heatmap(confusion_matrix(df_3['churn_class_number'], predictGB_2), annot=True, fmt='g', ax=ax);

# labels, title and ticks
ax.set_xlabel('Predicted labels');ax.set_ylabel('True labels'); 
ax.set_title('Confusion Matrix GB'); 
ax.xaxis.set_ticklabels(['0', '1']); ax.yaxis.set_ticklabels(['0', '1']);

In [ ]:
df_3

In [ ]:
# Plot da confusion matrix
# ax = plt.subplot()
# sns.heatmap(confusion_matrix(df_2['churn_class_number'], predictSVM_2), annot=True, fmt='g', ax=ax);

# labels, title and ticks
# ax.set_xlabel('Predicted labels');ax.set_ylabel('True labels'); 
# ax.set_title('Confusion Matrix SVM'); 
# ax.xaxis.set_ticklabels(['0', '1']); ax.yaxis.set_ticklabels(['0', '1']);

In [ ]:
# Plot da confusion matrix
# ax = plt.subplot()
# sns.heatmap(confusion_matrix(df_2['churn_class_number'], predictKN_2), annot=True, fmt='g', ax=ax);

# labels, title and ticks
# ax.set_xlabel('Predicted labels');ax.set_ylabel('True labels'); 
# ax.set_title('Confusion Matrix KN'); 
# ax.xaxis.set_ticklabels(['0', '1']); ax.yaxis.set_ticklabels(['0', '1']);

In [ ]:
# Plot da confusion matrix
# ax = plt.subplot()
# sns.heatmap(confusion_matrix(df_2['churn_class_number'], predictNB_2), annot=True, fmt='g', ax=ax);

# labels, title and ticks
# ax.set_xlabel('Predicted labels');ax.set_ylabel('True labels'); 
# ax.set_title('Confusion Matrix NB'); 
# ax.xaxis.set_ticklabels(['0', '1']); ax.yaxis.set_ticklabels(['0', '1']);